In [1]:
# google drive mount

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595753 sha256=bc160bb0e6c43527a484ffb665240b26da31ed2613df3a9c1f2529eb6b3b8c0a
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 103.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hr3gi70a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hr3gi70a
     |████████████████████████████████| 129 kB 5.0 MB/s 
     |████████████████████████████████| 54.7 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 83.1 MB/s 
     |████████████████████████████████| 2.5 MB 51.7 MB/s 
     |████████████████████████████████| 6.7 MB 75.1 MB/s 
     |████████████████████████████████| 73 kB 2.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15708 sha256=cfb952c4dea26aefcfb013e2ade4730233b5a0752478d80599954df69eb5ff1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-y6zbc7f9/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: transformers
    Fo

In [ ]:
%cd '/gdrive/My Drive/KoBERT/checkpoint/'

/gdrive/My Drive/KoBERT/checkpoint


In [5]:
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert import get_onnx_kobert_model

onnx_path = get_onnx_kobert_model()
sess = onnxruntime.InferenceSession(onnx_path)

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

/content/.cache/kobert.onnx1.8.0.onnx[██████████████████████████████████████████████████]


ValueError: ignored

In [ ]:
# tokenizing
from torch.utils.data import Dataset, DataLoader

class EmotionDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, 
                                               max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair) 

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

In [ ]:
# classifier
import torch
from torch import nn

class EmotionClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 9,
               dr_rate = None,
               params = None):
    super(EmotionClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    
    _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), 
                          attention_mask=attention_mask.float().to(token_ids.device), return_dict=False)
    
    if self.dr_rate:
      out = self.dropout(pooler)
    
    return self.classifier(out)

In [ ]:
# test
import torch

# load model
model1 = torch.load('kobert_v2.pt', map_location=torch.device('cpu'))
model1.load_state_dict(torch.load('kobert_v2_state_dict.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
import numpy as np

def torch_to_onnx():

    model1.eval()

    max_len = 64
    batch_size = 32

    data =['안녕하세요 테스트 문장입니다', '0']
    dataset_new = [data]

    new_test = EmotionDataset(dataset_new, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(new_test, batch_size=batch_size, num_workers=4)

    # dummy input x = 모델의 input = 모델의 forward 함수를 통과하는 타입
    for token_ids, valid_length, segment_ids, label in test_dataloader:   
        x = token_ids, valid_length, segment_ids

    torch.onnx.export(
        model1,
        x,
        'model.onnx',
        export_params=True,
        # verbose=True,
        input_names=['token_ids', 'valid_length', 'segment_ids'],
        output_names=['output'],
        opset_version=13
    )

In [ ]:
max_len = 64
batch_size = 32

data =['안녕하세요 테스트 문장입니다', '0']
dataset_new = [data]

new_test = EmotionDataset(dataset_new, 0, 1, tok, vocab, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(new_test, batch_size=batch_size, num_workers=4)

for token_ids, valid_length, segment_ids, label in test_dataloader:   
    x = token_ids, valid_length, segment_ids
    y = model1(token_ids, valid_length, segment_ids)

print(x)
print(y)

(tensor([[   2, 3135, 5724, 7814, 4736, 2120, 7178, 7139,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]], dtype=torch.int32), tensor([9], dtype=torch.int32), tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32))
tensor([[ 3.8171, -2.0730, -0.8213, -0.1868, -0.6384, -0.4231,  3.3142, -0.6208,
         -0.9593]], grad_fn=<AddmmBackward>)


In [ ]:
torch_to_onnx()

In [ ]:
predict('안녕하세요 테스트 문장입니다')

kobert_v2 : [3.817, 0, 0, 0, 0, 0, 3.314, 0, 0] 중립


'중립'

In [ ]:
import onnx

from onnx import helper as h
from onnx import checker as ch
from onnx import TensorProto, GraphProto
from onnx import numpy_helper as nph

import numpy as np
from collections import OrderedDict

# from logger import log
# import typer


def make_param_dictionary(initializer):
    params = OrderedDict()
    for data in initializer:
        params[data.name] = data
    return params


def convert_params_to_int32(params_dict):
    converted_params = []
    for param in params_dict:
        data = params_dict[param]
        if data.data_type == TensorProto.INT64:
            data_cvt = nph.to_array(data).astype(np.int32)
            data = nph.from_array(data_cvt, data.name)
        converted_params += [data]
    return converted_params


def convert_constant_nodes_to_int32(nodes):
    """
    convert_constant_nodes_to_int32 Convert Constant nodes to INT32. If a constant node has data type INT64, a new version of the
    node is created with INT32 data type and stored.

    Args:
        nodes (list): list of nodes

    Returns:
        list: list of new nodes all with INT32 constants.
    """
    new_nodes = []
    for node in nodes:
        if (
            node.op_type == "Constant"
            and node.attribute[0].t.data_type == TensorProto.INT64
        ):
            data = nph.to_array(node.attribute[0].t).astype(np.int32)
            new_t = nph.from_array(data)
            new_node = h.make_node(
                "Constant",
                inputs=[],
                outputs=node.output,
                name=node.name,
                value=new_t,
            )
            new_nodes += [new_node]
        else:
            new_nodes += [node]

    return new_nodes


def convert_model_to_int32(model_path: str, out_path: str):
    """
    convert_model_to_int32 Converts ONNX model with INT64 params to INT32 params.\n

    Args:\n
        model_path (str): path to original ONNX model.\n
        out_path (str): path to save converted model.
    """
    # log.info("ONNX INT64 --> INT32 Converter")
    # log.info(f"Loading Model: {model_path}")
    # * load model.
    model = onnx.load_model(model_path)
    ch.check_model(model)
    # * get model opset version.
    opset_version = model.opset_import[0].version
    graph = model.graph
    # * The initializer holds all non-constant weights.
    init = graph.initializer
    # * collect model params in a dictionary.
    params_dict = make_param_dictionary(init)
    # log.info("Converting INT64 model params to INT32...")
    # * convert all INT64 aprams to INT32.
    converted_params = convert_params_to_int32(params_dict)
    # log.info("Converting constant INT64 nodes to INT32...")
    new_nodes = convert_constant_nodes_to_int32(graph.node)

    graph_name = f"{graph.name}-int32"
    # log.info("Creating new graph...")
    # * create a new graph with converted params and new nodes.
    graph_int32 = h.make_graph(
        new_nodes,
        graph_name,
        graph.input,
        graph.output,
        initializer=converted_params,
    )
    # log.info("Creating new int32 model...")
    model_int32 = h.make_model(graph_int32, producer_name="onnx-typecast")
    model_int32.opset_import[0].version = opset_version
    ch.check_model(model_int32)
    # log.info(f"Saving converted model as: {out_path}")
    onnx.save_model(model_int32, out_path)
    # log.info(f"Done Done London. 🎉")
    return

In [ ]:
convert_model_to_int32('model.onnx', 'model_32.onnx')

In [ ]:
import onnx

onnx_model = onnx.load('model_32.onnx')
onnx.checker.check_model(onnx_model)

In [ ]:
!pip install onnxruntime==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import onnxruntime

onnxruntime.__version__

'1.7.0'

In [ ]:
ort_session = onnxruntime.InferenceSession('model_32.onnx')

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(y), ort_outs[0], rtol=1e-03, atol=1e-05)

InvalidGraph: ignored